In [1]:
! $HOME/Documents/VirtualEnvs/keras/bin/activate && . /usr/local/bin/thisroot.sh

/bin/sh: /Users/adrianodiflorio/Documents/VirtualEnvs/keras/bin/activate: Permission denied


In [2]:
import ROOT
from ROOT import TFile,TH1,TH1F,TCanvas
from ROOT import RooFit
from ROOT.RooFit import Layout

Welcome to JupyROOT 6.10/08


#rootfile = "X4140_MuMuKK_KRe_MuRe_NoCuts_NP3.0_Alpha99_CW5.15-5.55.root"
rootfile = "X4140_MuMuKK_KRe_MuRe_ChiCuts_NP3.0_Alpha99_CW5.15-5.55.root"
rootfile ="../rootfiles/X4140_Base_KRe_MuRef_NoQual_Sidebands5-7_NP3.0_B0Cuts_CW5.15-5.55_BDCAdd.root"
rootfile = "../rootfiles/X4140_Base_Side4-6_AllData.root"
histname = "Xcand_histo_hlt8_cw_nonprompt_cosalpha"
histname = "X5568_Cand_Mass_Ref_NoMult"
histname = "CW_Mass_Signal_NoM"
#histname = "CW_Cand_Mass_NoM"
#histname = "Xcand_histo_DM_any_cw_nonprompt_cosalpha"
inputfile  = TFile(rootfile,"READ")
hist = inputfile.Get(histname)
c = TCanvas("canvas","canvas",1200,800) ;

In [4]:
hist.GetName()
hist.Rebin(10)
hist.GetBinWidth(2)

0.005

In [5]:
from ROOT import RooRealVar,RooAbsPdf,RooChebychev,RooExponential,RooGaussian,RooAbsPdf,RooPlot,RooAddPdf,RooDataHist,RooArgSet,RooArgList
from ROOT import kGreen,kRed,kBlack,kBlue,kDashed,kDotted,kMagenta
from ROOT.RooFit import Components,LineColor,LineStyle,Name,Normalization

In [6]:
massmin = 5.2
massmax = 5.55
massbins = (massmax - massmin)/0.005
mass = RooRealVar("mass","M(#mu#muKK)[GeV]",massmin,massmax)
mass.setBins(int(massbins))

In [7]:
mean = RooRealVar("mean","mean of gaussian",5.38,5.31,5.41);
sigma1 = RooRealVar("sigma1","width of gaussian1",0.002,0.0005,0.05);
sigma2 = RooRealVar("sigma2","width of gaussian2",0.004,0.004,0.01);

a0 = RooRealVar("a0","a0",0.001,-1.,1.)
a1 = RooRealVar("a1","a1",0.001,-0.5,0.5)
a2 = RooRealVar("a2","a2",-0.00001,-2.,2.)
a3 = RooRealVar("a3","a3",-0.000001,-0.1,0.1)
a4 = RooRealVar("a4","a4",-0.000001,-2.,2.)
a5 = RooRealVar("a5","a5",-0.000001)
a6 = RooRealVar("a6","a6",-0.000001,-0.01,0.01)

aset = RooArgList(a0,a1,a2)#,a3)
gaussFrac = RooRealVar("sig1frac","fraction of component 1 in signal",0.3,0.0,1.0)
nSig = RooRealVar("nSig","nSig",55000,0.,10E6)
nBkg = RooRealVar("nBkg","nBkg",55000,0.,10E6)

In [8]:
cheb = RooChebychev("cheb","Background",mass,aset)
gauss1 = RooGaussian("gauss1","gaussian PDF 1",mass,mean,sigma1)
gauss2 = RooGaussian("gauss2","gaussian PDF 2",mass,mean,sigma2)

model  = RooAddPdf("model","g1+g2",gauss1,gauss2,gaussFrac)
tot = RooAddPdf("tot","g1+g2+cheb",RooArgList(model,cheb),RooArgList(nSig,nBkg))

In [9]:
masslist = RooArgList(mass)
dh = RooDataHist("dh","dh",masslist,hist);

[#1] INFO:DataHandling -- RooDataHist::adjustBinning(dh): fit range of variable mass expanded to nearest bin boundaries: [5.2,5.55] --> [5.2,5.55]


In [10]:
tot.fitTo(dh)

[#1] INFO:Minization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (gauss1,gauss2,cheb)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 a0           1.00000e-03  2.00000e-01   -1.00000e+00  1.00000e+00
     2 a1           1.00000e-03  1.00000e-01   -5.00000e-01  5.00000e-01
     3 a2          -1.00000e-05  4.00000e-01   -2.00000e+00  2.00000e+00
     4 mean         5.38000e+00  1.00000e-02    5.31000e+00  5.41000e+00
     5 nBkg         5.50000e+04  2.75000e+04    0.00000e+00  1.00000e+07
     6 nSig         5.50000e+04  2.75000e+04    0.00000e+00  1.00000e+07
     7 sig1frac     3.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
  

In [17]:
massFrame = mass.frame()
massFrame.SetTitle("B_{0}^{s} signal")
massFrame.GetYaxis().SetTitleOffset(1.35)
dh.plotOn(massFrame)
tot.plotOn(massFrame)
gauss2.plotOn(massFrame,LineColor(kRed),LineStyle(kDashed),Normalization(gaussFrac.getValV()*nSig.getValV()/(nBkg.getValV()+nSig.getValV())))
gauss1.plotOn(massFrame,LineColor(kGreen),LineStyle(kDashed),Normalization((1.0-gaussFrac.getValV())*nSig.getValV()/(nBkg.getValV()+nSig.getValV())))
cheb.plotOn(massFrame,LineColor(kMagenta),LineStyle(kDotted),Normalization(nBkg.getValV()/(nBkg.getValV()+nSig.getValV())))
tot.paramOn(massFrame,Layout(0.60,0.99,0.9))
massFrame.Draw()

In [18]:
plotname = "b0test_param_" + histname
c.SaveAs(plotname  + ".png")
c.SaveAs(plotname  + ".eps")
c.SaveAs(plotname  + ".pdf")

Info in <TCanvas::Print>: png file b0test_param_CW_Mass_Signal_NoM.png has been created
Info in <TCanvas::Print>: eps file b0test_param_CW_Mass_Signal_NoM.eps has been created
Info in <TCanvas::Print>: pdf file b0test_param_CW_Mass_Signal_NoM.pdf has been created
